<a href="https://colab.research.google.com/github/lw-miles24/CompModNervSys-final/blob/main/simulate_BallAndStick_FXS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize session

Install neuron as usual.

In [1]:
%pip install neuron # only need to run this cell once to install
                    # neuron in the local jupyter environment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 23.3 MB/s eta 0:00:00


Mount the rest of the model files from the github repository.

In [2]:
repo_name = 'CompModNervSys-final'
if 'google.colab' in str(get_ipython()):
    import os
    if not os.path.exists(repo_name):
        !git clone https://github.com/lw-miles24/{repo_name}.git # downloads repository into our Google colab session's file system

    os.chdir(repo_name) # Changing working directory to downloaded repository

# Compile mechanisms
!nrnivmodl mechanisms

Cloning into 'CompModNervSys-final'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 98 (delta 41), reused 41 (delta 13), pack-reused 0
Receiving objects: 100% (98/98), 560.33 KiB | 7.47 MiB/s, done.
Resolving deltas: 100% (41/41), done.
/content/CompModNervSys-final
Mod files: "mechanisms/mechanisms/kv.mod" "mechanisms/mechanisms/na.mod"

Creating 'x86_64' directory for .o files.

 -> NMODL ../mechanisms/kv.mod
 -> Compiling mod_func.cpp
 -> NMODL ../mechanisms/na.mod
Translating kv.mod into /content/CompModNervSys-final/x86_64/kv.c
Thread Safe
 -> Compiling kv.c
Translating na.mod into /content/CompModNervSys-final/x86_64/na.c
Thread Safe
 -> Compiling na.c
 => LINKING shared library ./libnrnmech.so
 => LINKING executable ./special LDFLAGS are:    -pthread
Successfully created x86_64/special


Import the packages we'll be using.

In [3]:
from neuron import h # all NEURON hoc functions are accessed through h
from neuron import gui # if you're running this notebook in a local environment (instead of on google colab), launches the GUI
h.load_file('stdrun.hoc') # loads neuron standard hoc library, not always necessary
import numpy as np
import matplotlib.pyplot as plt
import plotly
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()


# Dendritic AIS model

Create the Ball and Stick neuron model

In [ ]:
from ballandstick import BallAndStick

ais_mode = 'dend' # set to either 'soma' or 'dend'
ais_loc = 0.028 # if ais_mode is 'dend', sets how far along the basal dendrite to attach the AIS
               # benchmark values from Hodapp et al. 2022, ranging 0 to 1 (along 200 µm basal dendrite)
               # mean axonal distance: 0.028 (5.7 µm)
               # 95th percentile:      0.06   (12 µm)
ais_length = 25.9 # µm; from Booker et al. 2020
cell_WT_dAIS = BallAndStick(0,ais_mode,ais_loc,ais_length=ais_length)

h.topology()
# Time vector
t_vec = h.Vector().record(h._ref_t)
# Simulation parameter settings
h.celsius = 37 # degC
h.steps_per_ms = 1/0.025
h.dt = 0.025 # ms
h.tstop = 20 # ms
h.v_init = -69 # mV

# Stimulation
iclamp = h.IClamp(cell_dAIS.soma(0.5))
iclamp.delay = 5 # ms
iclamp.dur = 5 # ms
iclamp.amp = 1 # ms
iclamp_i = h.Vector().record(iclamp._ref_i)


|-----|       BallAndStick[0].soma(0-1)
       `--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|       BallAndStick[0].dend[0](0-1)
             `--------------------|       BallAndStick[0].ais_prox(0-1)
                                   `--------------------|       BallAndStick[0].ais_dist(0-1)
                                                         `------------------------|       BallAndStick[0].axon(0-1)
       `----------|       BallAndStick[0].dend[1](0-1)
    `--------------------|       BallAndStick[0].apic[0](0-1)
                          `----|       BallAndStick[0].apic[1](0-1)
                          `----|       BallAndStick[0].apic[2](0-1)



Add excitatory synapse to basal dendritic branch with AIS (`dend[0]`), distal to AIS. Add inhibitory synapse to soma.

In [ ]:
# Add excitatory synapse to basal dendrite
exc_syn_loc = 0.438 # 0 to 1, sets how far along basal dendrite excitatory synapse is added
                    # 0.438 = 87.6 µm along 200 µm basal dendrite (average stim
                    # location used in Hodapp et al. 2022 simulations)
exc_syn = h.Exp2Syn(cell_dAIS.dend[0](exc_syn_loc))
exc_syn.tau1 = 0.175 # ms - rise time (taken from Hodapp 2022 supp materials)
exc_syn.tau2 = 5 # ms - decay time
exc_syn.e = 0 # mV - eversal potential

# Add inhibitory synapse to soma
inh_syn = h.Exp2Syn(cell_dAIS.soma(0.5))
inh_syn.tau1 = 1.6 # ms
inh_syn.tau2 = 11 # ms
inh_syn.e = -75 # mV

# synaptic current recordings
exc_syn_i = h.Vector().record(exc_syn._ref_i)
exc_syn_g = h.Vector().record(exc_syn._ref_g)
inh_syn_i = h.Vector().record(inh_syn._ref_i)
inh_syn_g = h.Vector().record(inh_syn._ref_g)

Make `NetStim`s to drive synapses and `NetCons` to connect `NetStim`s to the synapses.  

In [ ]:
# NetStim to simulate excitatory presynaptic spikes
estim = h.NetStim()
estim.number = 1
estim.interval = 10 # ms - interval between spikes
estim.start = 5 # ms
estim.noise = 0

# NetCon to connect excitatory presynaptic spikes to excitatory postsynaptic conductance
nc_estim = h.NetCon(estim,exc_syn)
nc_estim.delay = 1 # ms - synaptic delay
nc_estim.weight[0] = 1 # µS - peak conductance

# NetStim to simulate inhibitory presynaptic spikes
istim = h.NetStim()
istim.number = 1
istim.interval = 10 # ms - interval between spikes
istim.start = 5 # ms
istim.noise = 0

# NetCon to connect inhibitory presynaptic spikes to excitatory postsynaptic conductance
nc_istim = h.NetCon(istim,inh_syn)
nc_istim.delay = 1 # ms - synaptic delay
nc_istim.weight[0] = 1 # µS - peak conductance

**Opt. 1** Define helper function to detect if the cell fired an AP. If it did, also determine at what voltage threshold this occurred.

**Opt. 2** Define helper function to detect if the cell fired an AP, checking if it happened in the soma & AIS (1.0) vs. AIS only (0.5) vs. neither (0).

In [ ]:
# def what_v_fire(v_vec,t_vec,dt=h.dt,threshold=30):
    # v_vec = vector of AIS voltage, used to determine if an AP fired
    # t_vec = time vector
    # dt = size of time step (ms)
    # threshold = rate of change of Vm for AP (10 mV/ms taken from Hodapp et al. 2022)
    # returns 1 if Vm gets over threshold mV
#    for i in np.arange(1,len(v_vec)):
#        if (v_vec[i] - v_vec[i-1])/dt > threshold:
#            return v_vec[i],t_vec[i]
#    return False,0

In [ ]:
def what_v_fire(v_vec_AIS,v_vec_soma,t_vec,threshold=0):
    # returns 1 if Vm in AIS & soma exceeds threshold; 0.5 if only AIS exceeds; 0 if neither exceed
    # v_vec_AIS = vector of AIS voltage
    # v_vec_soma = vector of soma voltage
    # t_vec = time vector
    # dt = size of time step (ms)
    # threshold = mV
    # for i in np.arange(1,len(t_vec)):
    #    if v_vec_AIS[i] > threshold:
    #        if v_vec_soma[i] > threshold:
    #            return 1
    #        else:
    #            return 0.5
    # return 0
    for i in np.arange(0,len(t_vec)):
        if v_vec_AIS[i] > threshold:
            return v_vec_soma.max()
    return False

Simulate a range of excitatory and inhibitory weights from Hodapp et al. 2022.

In [ ]:
exc_range = np.linspace(0,0.12,25) #13
inh_range = np.linspace(0,0.08,17) #9
iclamp.amp=0

dAIS_fire_matrix = np.zeros((len(exc_range),len(inh_range)))

for e in np.arange(0,len(exc_range)):
    for i in np.arange(0,len(inh_range)):
        nc_estim.weight[0] = exc_range[e]
        nc_istim.weight[0] = inh_range[i]
        h.run()
        ifAP_soma_v_max = what_v_fire(cell_dAIS.ais_v,cell_dAIS.soma_v,t_vec,threshold=0)
        if ifAP_soma_v_max:
            dAIS_fire_matrix[e,i] = ifAP_soma_v_max
        else:
            dAIS_fire_matrix[e,i] = np.nan


Plot on heatmap.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(z=dAIS_fire_matrix,x=inh_range*1e3,y=exc_range*1e3))
fig.update_xaxes(title_text="Inhibitory weight (nS)")
fig.update_yaxes(title_text="Excitatory weight (nS)")
fig.update_traces(zmax=25,zmin=-45,colorbar_title_text='peak soma voltage (mV)',colorbar_title_side='right',selector=dict(type='heatmap'))
fig.show()

# Somatic AIS model

Create the Ball and Stick neuron model

In [ ]:
from ballandstick import BallAndStick

ais_mode = 'soma' # set to either 'soma' or 'dend'
ais_loc = 0 # if ais_mode is 'dend', sets how far along the basal dendrite to attach the AIS
               # benchmark values from Hodapp et al. 2022, ranging 0 to 1 (along 200 µm basal dendrite)
               # mean axonal distance: 0.028 (5.7 µm)
               # 95th percentile:      0.06   (12 µm)
cell_sAIS = BallAndStick(0,ais_mode,ais_loc)

h.topology()
# Time vector
t_vec = h.Vector().record(h._ref_t)
# Simulation parameter settings
h.celsius = 37 # degC
h.steps_per_ms = 1/0.025
h.dt = 0.025 # ms
h.tstop = 20 # ms
h.v_init = -69 # mV

# Stimulation
iclamp = h.IClamp(cell_sAIS.soma(0.5))
iclamp.delay = 5 # ms
iclamp.dur = 5 # ms
iclamp.amp = 1 # ms
iclamp_i = h.Vector().record(iclamp._ref_i)


|-----|       BallAndStick[0].soma(0-1)
       `--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|       BallAndStick[0].dend[0](0-1)
             `--------------------|       BallAndStick[0].ais_prox(0-1)
                                   `--------------------|       BallAndStick[0].ais_dist(0-1)
                                                         `------------------------|       BallAndStick[0].axon(0-1)
       `----------|       BallAndStick[0].dend[1](0-1)
    `--------------------|       BallAndStick[0].apic[0](0-1)
                          `----|       BallAndStick[0].apic[1](0-1)
                          `----|       BallAndStick[0].apic[2](0-1)
|-----|       BallAndStick[0].soma(0-1)
       `-----------------------------------------------------------------------------------------------------------------------------------

Add excitatory synapse to basal dendritic branch with AIS (`dend[0]`), distal to AIS. Add inhibitory synapse to soma.

In [ ]:
# Add excitatory synapse to basal dendrite
exc_syn_loc = 0.438 # 0 to 1, sets how far along basal dendrite excitatory synapse is added
                    # 0.438 = 87.6 µm along 200 µm basal dendrite (average stim
                    # location used in Hodapp et al. 2022 simulations)
exc_syn = h.Exp2Syn(cell_sAIS.dend[0](exc_syn_loc))
exc_syn.tau1 = 0.175 # ms - rise time (taken from Hodapp 2022 supp materials)
exc_syn.tau2 = 5 # ms - decay time
exc_syn.e = 0 # mV - eversal potential

# Add inhibitory synapse to soma
inh_syn = h.Exp2Syn(cell_sAIS.soma(0.5))
inh_syn.tau1 = 1.6 # ms
inh_syn.tau2 = 11 # ms
inh_syn.e = -75 # mV

# synaptic current recordings
exc_syn_i = h.Vector().record(exc_syn._ref_i)
exc_syn_g = h.Vector().record(exc_syn._ref_g)
inh_syn_i = h.Vector().record(inh_syn._ref_i)
inh_syn_g = h.Vector().record(inh_syn._ref_g)

Make `NetStim`s to drive synapses and `NetCons` to connect `NetStim`s to the synapses.  

In [ ]:
# NetStim to simulate excitatory presynaptic spikes
estim = h.NetStim()
estim.number = 1
estim.interval = 10 # ms - interval between spikes
estim.start = 5 # ms
estim.noise = 0

# NetCon to connect excitatory presynaptic spikes to excitatory postsynaptic conductance
nc_estim = h.NetCon(estim,exc_syn)
nc_estim.delay = 1 # ms - synaptic delay
nc_estim.weight[0] = 1 # µS - peak conductance

# NetStim to simulate inhibitory presynaptic spikes
istim = h.NetStim()
istim.number = 1
istim.interval = 10 # ms - interval between spikes
istim.start = 5 # ms
istim.noise = 0

# NetCon to connect inhibitory presynaptic spikes to excitatory postsynaptic conductance
nc_istim = h.NetCon(istim,inh_syn)
nc_istim.delay = 1 # ms - synaptic delay
nc_istim.weight[0] = 1 # µS - peak conductance

**Opt. 1** Define helper function to detect if the cell fired an AP. If it did, also determine at what voltage threshold this occurred.

**Opt. 2** Define helper function to detect if the cell fired an AP, checking if it happened in the soma & AIS (1.0) vs. AIS only (0.5) vs. neither (0).

In [ ]:
# def what_v_fire(v_vec,t_vec,dt=h.dt,threshold=30):
    # v_vec = vector of AIS voltage, used to determine if an AP fired
    # t_vec = time vector
    # dt = size of time step (ms)
    # threshold = rate of change of Vm for AP (10 mV/ms taken from Hodapp et al. 2022)
    # returns 1 if Vm gets over threshold mV
#    for i in np.arange(1,len(v_vec)):
#        if (v_vec[i] - v_vec[i-1])/dt > threshold:
#            return v_vec[i],t_vec[i]
#    return False,0

In [ ]:
def what_v_fire(v_vec_AIS,v_vec_soma,t_vec,threshold=0):
    # returns 1 if Vm in AIS & soma exceeds threshold; 0.5 if only AIS exceeds; 0 if neither exceed
    # v_vec_AIS = vector of AIS voltage
    # v_vec_soma = vector of soma voltage
    # t_vec = time vector
    # dt = size of time step (ms)
    # threshold = mV
    # for i in np.arange(1,len(t_vec)):
    #    if v_vec_AIS[i] > threshold:
    #        if v_vec_soma[i] > threshold:
    #            return 1
    #        else:
    #            return 0.5
    # return 0
    for i in np.arange(0,len(t_vec)):
        if v_vec_AIS[i] > threshold:
            return v_vec_soma.max()
    return False

Simulate a range of excitatory and inhibitory weights from Hodapp et al. 2022.

In [ ]:
exc_range = np.linspace(0,0.12,25) #13
inh_range = np.linspace(0,0.08,17) #9
iclamp.amp=0

sAIS_fire_matrix = np.zeros((len(exc_range),len(inh_range)))

for e in np.arange(0,len(exc_range)):
    for i in np.arange(0,len(inh_range)):
        nc_estim.weight[0] = exc_range[e]
        nc_istim.weight[0] = inh_range[i]
        h.run()
        ifAP_soma_v_max = what_v_fire(cell_sAIS.ais_v,cell_sAIS.soma_v,t_vec,threshold=0)
        if ifAP_soma_v_max:
            sAIS_fire_matrix[e,i] = ifAP_soma_v_max
        else:
            sAIS_fire_matrix[e,i] = np.nan


Plot on heatmap.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(z=sAIS_fire_matrix,x=inh_range*1e3,y=exc_range*1e3))
fig.update_xaxes(title_text="Inhibitory weight (nS)")
fig.update_yaxes(title_text="Excitatory weight (nS)")
fig.update_traces(zmax=25,zmin=-45,colorbar_title_text='peak soma voltage (mV)',colorbar_title_side='right',selector=dict(type='heatmap'))
fig.show()

# Single Traces

Define helper function to convert between `x` values (normalized distance along a cable) and the segment index (a number between `0` and `nseg-1`) within the dendrite.

In [ ]:
def xind(vec,val):
    # finds closest value to x in x_vals vector, outputs index
    # if x>1, outputs index for x = 1, and if x<0, outputs index for x = 0
    return np.argmin(np.abs(vec-val))

dend0_x_vals = np.array([seg.x for seg in cell_sAIS.dend[0]])
dend1_x_vals = np.array([seg.x for seg in cell_sAIS.dend[1]])

Set amplitude of current pulse, excitatory synapse, and/or inhibitory synapse, run, and plot

In [ ]:
nc_estim.weight[0] = 0.050
nc_istim.weight[0] = 0.05
iclamp.amp=0
h.run()

fig = plt.figure(figsize=(5,6))
ax0 = fig.add_subplot(211) # soma voltage
ax0.plot(t_vec,cell_sAIS.soma_v,label='soma')
# Uncomment to plot other voltages
ax0.plot(t_vec,cell_sAIS.ais_v,label='Distal AIS')
ax0.plot(t_vec,cell_sAIS.dend0_vs[xind(dend0_x_vals,0.5)],label='dend[0](0.5)')
ax0.plot(t_vec,cell_sAIS.dend1_vs[xind(dend1_x_vals,0.5)],label='dend[1](0.5)',ls='--')
# ax0.legend()
ax0.set_ylim(-80,40)
ax0.set_ylabel('Vm (mV)')
ax0.set_title('Voltage')
ax0.legend()
# ax1 = fig.add_subplot(312) # injected current
# ax1.plot(t_vec,iclamp_i)
# ax1.set_ylabel('Current (nA)')
# ax1.set_title('Somatic current injection')
ax2 = fig.add_subplot(212)
ax2.plot(t_vec,exc_syn_g,label='Excitatory synapse')
ax2.plot(t_vec,inh_syn_g,label='Inhibitory synapse')
ax2.set_ylabel('Conductance (uS)')
# ax2.plot(t_vec,exc_syn_i,label='Excitatory synapse')
# ax2.plot(t_vec,inh_syn_i,label='Inhibitory synapse')
ax2.set_xlabel('time (ms)')
ax2.legend()
ax2.set_title('Synaptic currents')
fig.tight_layout()

In [ ]:
diffs = []
range_l = 250
range_h = 360

for i in np.arange(range_l,range_h):
    diffs.append((np.array(cell_sAIS.ais_v)[i]-np.array(cell_sAIS.ais_v)[i-1])/(h.dt))

diffs = np.array(diffs)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(np.array(t_vec)[range_l:range_h],np.array(cell_sAIS.ais_v)[range_l:range_h])
ax.plot(np.array(t_vec)[range_l:range_h],diffs*1e-1)